In [1]:
import pandas as pd
import requests
import json
import pandasql as ps

# API

Receiving applications from the government's API to receive a price list for all vehicles


In [2]:
# קבלת הנתונים מאי פי אי של הממשלה לקבל מחירון לכל הרכבים
url = "https://data.gov.il/api/3/action/datastore_search?resource_id=39f455bf-6db0-4926-859d-017f34eacbcb&limit=5000"
response = requests.get(url)
response_j = response.content.decode('utf-8')
response_d = json.loads(response_j)

### Arranging the data
  Creating a price list table

In [3]:
#סידור הנתונים שקבלנו כי לא הצלחנו לקרא אותם דרך גייסון אז נקרא דרך ניקוי נתונים על ידי פונקציות מחרוזת
df_T=(((str(response_d).split("'records': ["))[1]).split("], 'fields'"))[0]
x = df_T.split("}, {")#פיצול הסטרינג לפי סוגריים לקבלת שורות
df = pd.DataFrame(columns = ['tozeret_nm','kinuy_mishari', 'shnat_yitzur', 'mehir'])
print(x[0])
#יצירת טבלת המחירון
for i in range(0,len(x)):#ריצה על כל השורות שקבלנו מעלה
    y=x[i].split(', ')#פיצול השורה לפי פסיקים- פיצרים/עמודות
    df = df.append({'tozeret_nm' : y[5].replace("'tozeret_nm': ",'').replace("'",''), 
                    'kinuy_mishari' : y[10].replace("'kinuy_mishari': ",'').replace("'",''), 
                    'shnat_yitzur' : y[8].replace("'shnat_yitzur': ",'').replace("'",''), 
                    'mehir' : y[9].replace("'mehir': ",'').replace("'",'')}, 
                ignore_index = True)
df

{'_id': 1, 'semel_yevuan': 102, 'shem_yevuan': 'דוד לובינסקי', 'sug_degem': 'M', 'tozeret_cd': 1365, 'tozeret_nm': 'סיטרואן אנגליה', 'degem_cd': 7, 'degem_nm': 'VFEHTM', 'shnat_yitzur': 2022, 'mehir': 215990, 'kinuy_mishari': 'JUMPY'


,tozeret_nm,kinuy_mishari,shnat_yitzur,mehir
0,סיטרואן אנגליה,JUMPY,2022,215990
1,סיטרואן אנגליה,JUMPY,2021,215990
2,סיטרואן אנגליה,JUMPY,2021,255990
3,סיטרואן אנגליה,JUMPY,2022,255990
4,סיטרואן אנגליה,JUMPY,2021,253990
...,...,...,...,...
4995,אאודי,A8,2017,885100
4996,אאודי,A5,2018,434800
4997,אאודי,A3 SPORTBACK,2018,240000
4998,אסטון מרטין,DB11,2018,1835000


In [4]:
dfNew = ps.sqldf("select tozeret_nm,kinuy_mishari,shnat_yitzur,avg(mehir) as mehir from df group by tozeret_nm,kinuy_mishari,shnat_yitzur")#בניית טבלה על ידי שאילתא

dfNew.to_excel("cars_pricing.xlsx")
dfNew

,tozeret_nm,kinuy_mishari,shnat_yitzur,mehir
0,"""טסלה ארהב""",MODEL 3,2020,322200.000000
1,"""טסלה ארהב""",MODEL 3,2021,279491.105263
2,"""יונדאי צכיה""",I30,2016,116442.857143
3,"""יונדאי צכיה""",I30,2017,123416.666667
4,"""יונדאי צכיה""",I30,2018,130900.000000
...,...,...,...,...
1530,רנו צרפת,TRAFIC,2019,219995.000000
1531,רנו צרפת,TRAFIC,2020,232990.000000
1532,רנו צרפת,TRAFIC,2021,236990.000000
1533,רנו צרפת,TRAFIC,2022,240990.000000
